In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk import pos_tag, word_tokenize
from string import punctuation
from sklearn import preprocessing, linear_model, metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import itertools
%matplotlib inline

In [2]:
df = pd.read_csv('Hotel_Reviews.csv')

In [3]:
len(df)

414737

In [4]:
df['Full_Review'] = df['Positive_Review'] + ' and ' + df['Negative_Review']

In [5]:
def cleanme(txt):
    sent = txt.lower()
    wrds = word_tokenize(sent)
    clwrds = [w for w in wrds if not w in stopwords.words('english')]
    ln = len(clwrds)
    pos = pd.DataFrame(pos_tag(wrds))
    pos = " ".join(list(pos[pos[1].str.contains("JJ")].iloc[:,0]))
    rt = [ln, " ".join(clwrds), pos]
    return(rt)

In [6]:
tmp = list()
for i in range(len(df)):
    tmp.append(cleanme(df.iloc[i,:]['Full_Review']))
tmp = pd.DataFrame(tmp)
tmp.columns = ['reviewlen', 'cleanrev', 'adjreview']

In [7]:
x1 = tmp['cleanrev']
x2 = tmp['adjreview']
y = df["Reviewer_Score"]

In [8]:
vect = TfidfVectorizer(ngram_range = (1,3))
x_vect1 = vect.fit_transform(x1)
x_vect2 = vect.fit_transform(x2)

In [11]:
def LinReg(x, y):
    cls = linear_model.LinearRegression()
    cls.fit(x,y)
    prediction= cls.predict(x)
    print('Co-efficient of linear regression',cls.coef_)
    print('Intercept of linear regression model',cls.intercept_)
    print('Mean Square Error', metrics.mean_squared_error(y, prediction))

In [13]:
LinReg(x_vect1, y)

Co-efficient of linear regression [-0.69074816 -0.25124314 -0.11778368 ...  0.13344441  0.13344441
  0.13344441]
Intercept of linear regression model 8.394804177105007
Mean Square Error 0.06452627653872954


In [14]:
LinReg(x_vect2, y)

Co-efficient of linear regression [-1.69778245 -1.69778245 -1.69778245 ... -0.23489912 -0.23489912
  0.02802873]
Intercept of linear regression model 8.394804177105007
Mean Square Error 0.5935256708855031


In [11]:
from sklearn.preprocessing import PolynomialFeatures

In [13]:
polynomial_features = PolynomialFeatures(degree = 2)
x_poly = polynomial_features.fit_transform(x_vect1)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

print('Co-efficient of linear regression',model.coef_)
print('Intercept of linear regression model',model.intercept_)
print('Mean Square Error', metrics.mean_squared_error(y, y_poly_pred))


KeyboardInterrupt: 